In [13]:
import warnings
warnings.filterwarnings(action='ignore')
import requests
import time
import folium
from pandas.io.json import json_normalize
import pandas as pd
from selenium import webdriver
driver = webdriver.Chrome("./chromedriver.exe")

1. 스타벅스 크롤링

In [33]:
url = 'https://www.starbucks.co.kr/store/getSidoList.do'
request = requests.post(url)
starbucks = request.json()
sido_List = {}
for sido in starbucks['list']:
    sido_List[sido['sido_cd']] =  sido['sido_nm']
print(sido_List)

{'01': '서울', '08': '경기', '02': '광주', '03': '대구', '04': '대전', '05': '부산', '06': '울산', '07': '인천', '09': '강원', '10': '경남', '11': '경북', '12': '전남', '13': '전북', '14': '충남', '15': '충북', '16': '제주', '17': '세종'}


In [27]:
sido_cd = input('구군코드를 검색하려는 시도코드를 입력하세요: ')
targetSite = 'https://www.starbucks.co.kr/store/getGugunList.do'
request = requests.post(targetSite, data={
    'sido_cd': sido_cd
})
gugunList = request.json()
gugun_List = {}
for gugun in gugunList['list']:
    gugun_List[gugun['gugun_cd']] = gugun['gugun_nm']
print(gugun_List)

구군코드를 검색하려는 시도코드를 입력하세요: 02
{'0201': '광산구 ', '0202': '남구', '0203': '동구', '0204': '북구', '0205': '서구'}


In [31]:
# 지역별 DT 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=BF1GQPNI9E'
request = requests.post(targetSite, data={
    'ins_lat': 37.5108295,
    'ins_lng': 127.0292881,
    'p_sido_cd': sido_cd,
    'p_gugun_cd': '',
    'in_biz_cd': '',
    'iend': 1600,
    'set_date': ''
})
starbucks = request.json()
starbucks_df = json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)

starbucks_map = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=11)
for index, data in starbucks_df_map.iterrows():
    if data['s_name'].find('DT') > 0:
        pop = folium.Popup(data['s_name'] + '점, 주소: ' + data['doro_address'], max_width=500)
        folium.Marker(location=[data['lat'], data['lot']], popup=pop,  icon=folium.Icon(color='green', icon='star')).add_to(starbucks_map)
starbucks_map.save('./starMap.html')
starbucks_map

2. 맥도날드 크롤링

In [18]:
url = "https://www.mcdonalds.co.kr/kor/store/list.do"
driver.get(url)

In [32]:
# 맥도날드 DT점 데이터
mac_data = []
for gu in gugun_List.values(): # 스타벅스 구군 데이터 재활용
    print(gu)
    try:
        driver.find_element_by_css_selector("#searchWord").clear()
    except:
        pass
    driver.find_element_by_css_selector("#searchWord").send_keys(gu)
    driver.find_element_by_css_selector("#searchForm > div > div > div > span:nth-child(2) > label").click()
    driver.find_element_by_css_selector("#searchForm > div > fieldset > div > button").click()
    time.sleep(1)
    
    dt_list=driver.find_elements_by_css_selector("#container > div.content > div.contArea > div > div > div.mcStore > table > tbody > tr")
    
    for data in dt_list:
        tmp = data.find_element_by_css_selector("td.tdName > dl > dt > strong > a")
        lat,lng = tmp.get_attribute("href")[19:-2].split(",")
        title = tmp.text
#         print(title,lat,lng)
        text = data.text
        
        if(sido_List.get(sido_cd) in text and "맥드라이브" in text): # sido_List.get(sido_cd) 시도 코드를 key 값으로함(스타벅스 코드 사용)
            mac_data.append({
                "title":title,
                "lat":lat,
                "lng":lng
            })
mac_df = pd.DataFrame(mac_data)
mac_df['lat'] = mac_df['lat'].astype(float) # 위도 => float로 변경
mac_df['lng'] = mac_df['lng'].astype(float) # 경도 => float로 변경
mac_map = folium.Map(location=[mac_df['lat'].mean(), mac_df['lng'].mean()], zoom_start=11)
for index, data in mac_df.iterrows():
    pop = folium.Popup(data['title'] + '점', max_width=500)
    folium.Marker(location=[data['lat'], data['lng']], popup=pop, icon=folium.Icon(color='red', icon='star')).add_to(mac_map)
mac_map

광산구 
남구
동구
북구
서구
